# Neighborhood Computations

In [1]:
# For debugging purpose
import sys
print(sys.path)

['/Users/kimhe/miniconda3/lib/python311.zip', '/Users/kimhe/miniconda3/lib/python3.11', '/Users/kimhe/miniconda3/lib/python3.11/lib-dynload', '', '/Users/kimhe/.local/lib/python3.11/site-packages', '/Users/kimhe/miniconda3/lib/python3.11/site-packages']


In [2]:
import numpy as np
import igl
import meshplot
import scipy as sp

In [3]:
bunny_v, bunny_f = igl.read_triangle_mesh("data/bunny.off")
cube_v, cube_f = igl.read_triangle_mesh("data/cube.obj")
sphere_v, sphere_f = igl.read_triangle_mesh("data/sphere.obj")

In [4]:
meshplot.plot(bunny_v, bunny_f, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

In [5]:
meshplot.plot(cube_v, cube_f, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [6]:
meshplot.plot(sphere_v, sphere_f, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

## Vertex-to-Face Relations

Given V and F, generate an adjacency list which contains, for each vertex, a
list of faces adjacent to it. The ordering of the faces incident on a vertex
does not matter. Your program should print out the vertex-to-face relations.

In [7]:
[VF, NI] = igl.vertex_triangle_adjacency(bunny_f,len(bunny_v))
print(VF)
print(NI)

[ 849  850  912 ... 1500 5112 5264]
[    0     5     9 ... 20884 20891 20898]


## Vertex-to-Vertex Relations

Given V and F, generate an adjacency list which contains, for each vertex, a
list of vertices connected with it. Two vertices are connected if there exists
an edge between them, i.e., if the two vertices appear in the same row of F. The
ordering of the vertices in each list does not matter.  Your program should
print out the vertex-to-vertex relations.

In [8]:
vtv_rel = igl.adjacency_list(bunny_f)
vtv_rel

[[3, 24, 308, 525, 542],
 [415, 549, 551, 596],
 [134, 287, 465, 497, 1308],
 [0, 246, 308, 510, 525, 543],
 [406, 1235, 1368, 1371, 1375],
 [792, 917, 1113, 3426, 3454, 3482],
 [484, 1130, 1163, 1179, 1191],
 [1236, 1267, 1352, 1381, 1382, 2146],
 [74, 1325, 1365, 1402, 1420],
 [280, 465, 1225, 1227, 1259],
 [11, 349, 451, 1319, 1524, 1537, 1565, 1671],
 [10, 12, 349, 1413, 1524, 1542, 1693],
 [11, 1542, 1693, 1724],
 [15, 238, 246, 266, 292, 526, 543],
 [156, 390, 440, 889],
 [13, 480, 503, 511, 526, 543],
 [17, 294, 324, 549, 551, 796, 1217],
 [16, 103, 324, 1217],
 [1547, 1598, 2014, 2034, 2053, 2070, 2085],
 [267, 343, 378, 506, 1436, 1450, 1451],
 [21, 34, 571, 1466, 1467, 1484, 1508],
 [20, 34, 56, 57, 58, 571, 594],
 [104, 295, 961, 1022, 1224, 1374],
 [350, 415, 740, 741, 1441, 1442],
 [0, 265, 291, 308, 542],
 [31, 36, 37, 38, 559, 568, 3324],
 [269, 337, 584, 622, 726, 771],
 [28, 1244, 1310, 1403, 1410, 1428],
 [27, 1236, 1410, 1428, 2136, 2146],
 [314, 1208, 1228, 1331, 13

## Shading

Meshplot requires per vertex normals, so we need to "explode" the mesh

In [9]:
# faces 
def explode_mesh(vertices, faces):
    exploded_vertices = np.empty((3*faces.shape[0], 3));
    exploded_faces = np.empty((faces.shape[0], 3), dtype="int32");
    for i in range(faces.shape[0]):
        for vi in range(3):
            exploded_vertices[3*i + vi, :] = vertices[faces[i, vi]]
            exploded_faces[i, vi] = 3*i + vi
    return exploded_vertices, exploded_faces

# exploded faces are sequential
def explode(per_face_n):
    exploded_n = np.empty((3*per_face_n.shape[0], 3));
    for i in range(per_face_n.shape[0]):
        for j in range(3):
            exploded_n[3*i + j, :] = per_face_n[i, :]
    
# Example usage:
original_vertices = np.array([[0, 0, 0], [1, 0, 0], [1, 1, 0], [0, 1, 0]])
original_faces = np.array([[0, 1, 2], [1, 3, 2]])
exploded_vertices, exploded_faces = explode_mesh(original_vertices, original_faces)
print("Exploded vertices:", exploded_vertices)
print("Exploded faces:", exploded_faces)

Exploded vertices: [[0. 0. 0.]
 [1. 0. 0.]
 [1. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 1. 0.]]
Exploded faces: [[0 1 2]
 [3 4 5]]


### Flat Shading

In [10]:
vec = np.array([0.57735027, 0.57735027, 0.57735027])

bunny_v_explode, bunny_f_explode = explode_mesh(bunny_v, bunny_f)
bunny_face_n = igl.per_face_normals(bunny_v_explode, bunny_f_explode, vec)
bunny_face_n_explode = explode(bunny_face_n)
p2_b = meshplot.plot(bunny_v_explode, bunny_f_explode, n=bunny_face_n_explode, shading={"flat": True})
# add normal to face
bc_b = igl.barycenter(bunny_v, bunny_f)
p2_b.add_lines(bc_b, bc_b + 0.01*bunny_face_n)

cube_v_explode, cube_f_explode = explode_mesh(cube_v, cube_f)
cube_face_n = igl.per_face_normals(cube_v_explode, cube_f_explode, vec)
cube_face_n_explode = explode(cube_face_n)
p2_c = meshplot.plot(cube_v_explode, cube_f_explode, n=cube_face_n_explode, shading={"flat": False})
# add normal to face
bc_c = igl.barycenter(cube_v, cube_f)
p2_c.add_lines(bc_c, bc_c + 0.5*cube_face_n)

sphere_v_explode, sphere_f_explode = explode_mesh(sphere_v, sphere_f)
sphere_face_n = igl.per_face_normals(sphere_v_explode, sphere_f_explode, vec)
sphere_face_n_explode = explode(sphere_face_n)
p2_s = meshplot.plot(sphere_v_explode, sphere_f_explode, n=sphere_face_n_explode, shading={"flat": False})
# add normal to face
bc_s = igl.barycenter(sphere_v, sphere_f)
p2_s.add_lines(bc_s, bc_s + 0.2*sphere_face_n)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

1

### Per-vertex Shading

In [11]:
# this is already per-vertex so no need to explode
bunny_n = igl.per_vertex_normals(bunny_v, bunny_f, 0)
p_b = meshplot.plot(bunny_v, bunny_f, n=bunny_n, shading={"flat": False})
# p_b.add_lines(bunny_v, bunny_v + 0.01*bunny_n)

cube_n = igl.per_vertex_normals(cube_v, cube_f, 0)
p_c = meshplot.plot(cube_v, cube_f, n=cube_n, shading={"flat": False})

sphere_n = igl.per_vertex_normals(sphere_v, sphere_f, 0)
p_s = meshplot.plot(sphere_v, sphere_f, n=sphere_n, shading={"flat": False})


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

### Per-corner Shading

In [12]:
# |dihedral angle| > 20 degrees --> crease
bunny_corner_n = igl.per_corner_normals(bunny_v, bunny_f, 80) 
bunny_v_explode, bunny_f_explode = explode_mesh(bunny_v, bunny_f)
p3_b = meshplot.plot(bunny_v_explode, bunny_f_explode, n=bunny_corner_n, shading={"flat": False})

cube_corner_n = igl.per_corner_normals(cube_v, cube_f, 80)
cube_v_explode, cube_f_explode = explode_mesh(cube_v, cube_f)
p3_c = meshplot.plot(cube_v_explode, cube_f_explode, n=cube_corner_n, shading={"flat": False})

sphere_corner_n = igl.per_corner_normals(sphere_v, sphere_f, 80)
sphere_v_explode, sphere_f_explode = explode_mesh(sphere_v, sphere_f)
p3_s = meshplot.plot(sphere_v_explode, sphere_f_explode, n=sphere_corner_n, shading={"flat": False})


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

## Connected Components

In [13]:
car_v, car_f = igl.read_triangle_mesh("data/car.off")
coffee_v, coffee_f = igl.read_triangle_mesh("data/coffeecup.off")

def plot_cc(v, f):
    vtv_rel = igl.adjacency_matrix(f)
    count_c, comp_c, comp_k = igl.connected_components(vtv_rel)
    # some color for different component
    texture = np.array([[.1,.1,.1],
                       [.7,.5,.7],
                       [.5,.1,.1],
                       [.1,.5,.1],
                       [.1,.1,.5],
                       [.5,.5,.5],
                       [.1,.5,.5],
                       [.5,.0,.5],
                       [.7,.5,.1],
                       [.1,.5,.7],
                       [.1,.7,.5]])
    v_color = np.empty((comp_c.shape[0], 3))
    for i in range(comp_c.shape[0]):
        comp = comp_c[i]
        v_color[i, :] = texture[comp, :]
    
    pcc_c = meshplot.plot(v, f, c=v_color, shading={"flat": True})

plot_cc(car_v, car_f)
plot_cc(coffee_v, coffee_f)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-7.194198…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-3.253649…

## A simple subdivision scheme

Add a new vertex at location m_f for each face f in F of the original mesh. 
The new vertex will be located at the midpoint of the face. Append the newly created vertices M = {m_f} to V to create a new set of vertices $V'' = [V; M]$. Add three new faces for each face f in order by connecting m_f with edges to the original 3 vertices of the face; we call the set of this newly created faces F''.

Move each vertex v of the old vertices V to a new position p by averaging v with the positions of its neighboring vertices in the original mesh. If v has valence n and its neighbors in the original mesh (V, F) are v_0, v_1, ..., v_n, then the update rule is:

$$ 
p = (1-a_n)v + \frac{a_n}{n} \sum_{i=0}^{n-1} v_i
$$

$$
a_n = \frac{4-2\cos(\frac{2\pi}n)}{9}
$$

In [20]:
def division(v, f):
    
    bc_s = igl.barycenter(v, f)
    # new_vs = np.array([v; bc_s])
    new_fs = np.array((f.shape[0]*3, 3))
    for i in range(f.shape[0]):
        l = bc_s.shape[0]
        face = np.array((l ));
        
    for i in range(v.shape[0]):
        i
    # n: number of neighbouring vertices
    an = (4 - 2 * np.cos(2*np.pi / n)) / 9
    p = (1-an)*v + an/n*sum

# division(sphere_v, sphere_f)
sphere_f.shape

(760, 3)